In [1]:
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.cluster import KMeans
import numpy as np
data = fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554
X, y = data["data"], data["target"]

stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in stratSplit.split(X, y):
    X_train = X[train_index[:2000]]
    X_test = X[test_index[:1000]]
    
    y_train = y[train_index[:2000]]
    y_test = y[test_index[:1000]]

In [2]:
X_train.shape

(2000, 784)

In [10]:
km = KMeans(n_clusters=50)
km.fit(X_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [13]:
import heapq
def getClosestPoints(X_train, cluster_center, perc=0.05):
    dists = X_train - cluster_center
    distnp = np.sum(np.power(dists,2),axis=1)
    numItems = int(X_train.shape[0]*perc)
    nsmallest = heapq.nsmallest(numItems, distnp)
    return np.where(np.isin(distnp,nsmallest)==True)[0], np.argmin(distnp)

y_train_new = -1*np.ones(X_train.shape[0])
for cluster_center in km.cluster_centers_:
    closest_x_per_points, closest_point = getClosestPoints(X_train, cluster_center, perc=0.03)
    y_train_new[closest_x_per_points] = y_train[closest_point]
    
propagated_labels = np.where(y_train_new!=-1)[0]

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(X_train[propagated_labels], y_train_new[propagated_labels])

lr.score(X_test, y_test.astype(float))

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.702